In [1]:
import utils.disaster_data_utils as disaster
import utils.speech_data_utils as speech

/Users/julius/repos/fundamentals_project/notebooks/disaster_data/utils/disaster_data_utils.py:70: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(result.shape[0] == len(country_year_pairs), "The number of rows in the feature matrix should be equal to the number of country-year pairs.")


# Ensure we use only year and country pairs that both datasets share, so we get exaclty the same length feature and target matrices.

In [2]:
# Prepare training data
df_disaster = disaster.build_clean_dataframe()
df_speech = speech.build_dataframe()

/Users/julius/repos/fundamentals_project/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:

speech_countries = set(df_speech["country"].unique())
speech_years = set(df_speech["year"].unique())
disaster_years = set(df_disaster["Year"].unique())
diaster_countries = set(df_disaster["Country"].unique())

In [4]:
# for the speech dataframes, create country year pairs
speech_country_year_pairs = []
for row in df_speech.iterrows():
    country = row[1]["country"]
    year = row[1]["year"]
    speech_country_year_pairs.append((country, year))
speech_country_year_pairs = set(speech_country_year_pairs)
    
# for the disaster dataframes, create country year pairs
disaster_country_year_pairs = []
for row in df_disaster.iterrows():
    country = row[1]["Country"]
    year = row[1]["Year"]
    disaster_country_year_pairs.append((country, year))
disaster_country_year_pairs = set(disaster_country_year_pairs)

   
   
shared_pairs = list(speech_country_year_pairs.intersection(disaster_country_year_pairs))


keywords = ["Co2", "climate change"]

# shared_pairs = shared_pairs

In [5]:
print(len(shared_pairs))
X_train = disaster.create_feature_train_matrix(country_year_pairs=shared_pairs)
print(X_train.shape)



10
Creating feature matrix for 10 country-year pairs)


/Users/julius/repos/fundamentals_project/.venv/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(10, 4)


In [6]:
T_train = speech.create_target_value_matrix(country_year_pairs=shared_pairs, keywords=keywords)


Creating feature matrix for 10 country-year pairs)
prepoccesing df
adding keyword counts column to df by matching keywords in stemmed stemmed text
adding year score column to df


In [7]:
T_train

array([[1.        ],
       [0.        ],
       [0.        ],
       [0.33333333],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [0.55555556],
       [0.        ]])

In [8]:
print(T_train.shape)
print(X_train.shape)
# # FIXME: Should be same size.
# assert(X_train.shape[0] == T_train[0])



(10, 1)
(10, 4)


In [9]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)
# TODO: fix Nan Values
model.fit(X_train, T_train)

LinearRegression()

In [10]:
# map iso code to countries
import pandas as pd
path = "../../data/CO2emissions.csv"
# create df
df = pd.read_csv(path)# Replace 'your_file.csv' with the actual path to your CSV file

# create dictionary with iso codes as keys and countries as values using Country and ISO 3166-1 alpha-3 columns
d = dict(zip(df['ISO 3166-1 alpha-3'], df["Country"]))

In [11]:
d

{'AFG': 'Afghanistan',
 'ALB': 'Albania',
 'DZA': 'Algeria',
 'AND': 'Andorra',
 'AGO': 'Angola',
 'AIA': 'Anguilla',
 'ATA': 'Antarctica',
 'ATG': 'Antigua and Barbuda',
 'ARG': 'Argentina',
 'ARM': 'Armenia',
 'ABW': 'Aruba',
 'AUS': 'Australia',
 'AUT': 'Austria',
 'AZE': 'Azerbaijan',
 'BHS': 'Bahamas',
 'BHR': 'Bahrain',
 'BGD': 'Bangladesh',
 'BRB': 'Barbados',
 'BLR': 'Belarus',
 'BEL': 'Belgium',
 'BLZ': 'Belize',
 'BEN': 'Benin',
 'BMU': 'Bermuda',
 'BTN': 'Bhutan',
 'BES': 'Bonaire, Saint Eustatius and Saba',
 'BIH': 'Bosnia and Herzegovina',
 'BWA': 'Botswana',
 'BRA': 'Brazil',
 'VGB': 'British Virgin Islands',
 'BRN': 'Brunei Darussalam',
 'BGR': 'Bulgaria',
 'BFA': 'Burkina Faso',
 'BDI': 'Burundi',
 'KHM': 'Cambodia',
 'CAN': 'Canada',
 'CPV': 'Cape Verde',
 'CAF': 'Central African Republic',
 'TCD': 'Chad',
 'CHL': 'Chile',
 'CHN': 'China',
 'CXR': 'Christmas Island',
 'COL': 'Colombia',
 'COM': 'Comoros',
 'COG': 'Congo',
 'COK': 'Cook Islands',
 'CRI': 'Costa Rica',
 